# Basic Programmming with [Dann5 d5o 2 library](https://github.com/voya-voja/d5o2)

In the [Leap factoring demo](https://cloud.dwavesys.com/learning/user/nebojsa_2evojinovic_40rogers_2ecom/notebooks/leap/demos/factoring/01-factoring-overview.ipynb), the D-Wave system can be used to factor a whole number by running a multiplication circuit in reverse. The factoring demo shows how can be solved a [constraint satisfaction problem (CSP)]( https://docs.ocean.dwavesys.com/en/stable/concepts/csp.html) on a QAC. 

In a way **Dann5 d5o programming framework** is an extension of python programming langue that provides a human friendly generalization of CSP implementation, which permits quantum programmers to use constructs such as data types definitions, expression and assignments to define problem to be solved on quantum computers, annealers or simulators via QUBO transformation.

This Jupyter Notebook demonstrates:
1. definition of Qwhole variables and type conversion to Qbin using Dann5 d5o library
2. how to define and solve quantum addition and multiplication assignments by using d5o quantum annealing simulator
3. a programmatic conversion of a basic problem statement into a QUBO binary quadratic model (BQM) for quantum annealing computer (QAC), 
4. submission of the QUBO to the D-Wave simulator and system, and retrieval of corresponding solution samples, and 
5. conversion of the retrieved solution samples into human readable form
6. introduces quantum expressions in d5o and compares them to quantum assignments

So, if you have [installed Dann5 d5o library](https://pypi.org/project/dann5/) and all prerequisites, lets start.

## Defining quantum variables using d5o 2

In [1]:
from dann5.d5o2 import Qwhole

> The line above allows definition of **Q** (quantum) **whole variables** in python.

To define a Q variable **a** as an unknown whole number with **4 Q bits in S**(uperposition) state, it is as simple as:

In [2]:
var = Qwhole(4, "a")

> Note that **'var'** is a python variable with a reverence to the defined Q whole variable **'a'**.

There is no reason that python and Q variable can't have the same name, like in the following line. **A python variable has a reference to a defined Q variable with the same name 'b'**.

In [3]:
b = Qwhole(3, "b")

> The Q variable 'b' is of **Qwhole type with U**(nknown) **value**, meaning that **at least one of its Qbits is in a S**(uperposition)** state**. In this definition of Q variable 'b', all 3 Qbits are in S state.

Both, 'a' and 'b' Q variables are defined as *Unknown*. HOwever, if we want to define a **deterministic Q whole variable** we specify its initialization value as in following definition:

In [4]:
A = Qwhole("A", 15)

- **NOTE**: Like in python, a Q whole variable 'A' is different from Q variable 'a', i.e. **d5o is case sensitive**.

> The Q variable 'A' is a 4 Qbit whole number with value 15, i.e. all its Qbits set to 1, i.e. binary 1111.

In [5]:
from dann5.d5o2 import Qbin
print('The binarry presentation of a deterministic Qwhole varaible {} is {}.'.format(A.toString(), Qbin(A).toString()))
print('The binary presentation of an unknown Qwhole varaible {} is {},'.format(var.toString(), Qbin(var).toString()))
print('... and of {} is {},'.format(b.toString(), Qbin(b).toString()))

The binarry presentation of a deterministic Qwhole varaible A/4:15/ is A/4b1111/.
The binary presentation of an unknown Qwhole varaible a/4:U/ is a/4bU/,
... and of b/3:U/ is b/3bU/,


## Defining and solving d5o 2 quantum assignment
We can now define a quantum assignment (Qassignment) by assign an addition quantum expression 'a' + 'b' to 'A':

In [6]:
aA = A.assign(var + b)

In python, we write the expression **'var' + 'b'**, as python 'var' variable is reference of a definition of Q whole variable 'a'.
- **NOTE**: all the quantum variables in an expression or assignment have to be of the same type.
> In this notebook we cover Qwhole variables, however, at present dann5.d5o2 supports definition of quantum expressions and assignments using variables of Qbit, Qbool, Qbin, Qwhole and Qint types.

In [7]:
aA

- **NOTE**: aA is an instance of a dann5.d5o2.QwholeAssignment class, which is a specilization of dann5.d5o2.Qassignment, with the following Qwhole addition assigned:

In [8]:
print(aA.toString())

A/5:15/ = (a/4:U/ + b/3:U/)


The *aA.toString()* method returns the quantum assignment, where each variable is presented as 
> *variable_name* ***/*** *#_of_q-bits* ***:*** *varaible_value* ***/***

- **NOTE**: Unknown quantum whole variables 'a' and 'b' have *variable_value = U*, while deterministic quantum whole variable 'A' has *variable_value = 15*.

We can see the Qbit level decomposition of the Q assignment, by setting *decomposed* argument to *True* in the *aA.toString()* method. The default value of *decomposed* argument is *False*.

In [9]:
print(aA.toString(True))


A0/1/ = a0/S/ ^ b0/S/

A1/1/ = a1/S/ + b1/S/ + #0/S/
#0/S/ = #[A0/1/]

A2/1/ = a2/S/ + b2/S/ + #1/S/
#1/S/ = #[A1/1/]

A3/1/ = a3/S/ ^ #2/S/
#2/S/ = #[A2/1/]

A4/0/ = #[A3/1/]



The *aA.toString(True)* method returns line by line Q bit level assignments corresponding to the original quantum assignment, where each Qbit variable is presented as 
> *variable_name #_of_q-bit_level* ***/*** *q-bit_varaible_value* ***/***

- **NOTE**: The q-bit variables, like 'a0' or 'b2' of unknown quantum variables 'a' and 'b', are in superposition state, i.e. their *q-bit_variable_value = S*.

- **NOTE**: As a result of quantum addition of Qwhole variables with 4 Qbits ('a') and 3 Qbits ('b') can have a result with 5 Qbits, so the Q assignment result 'A' is extended by adding an additional Qbit 'A4' with a value assigned according to its type:
    - A deterministic Qwhole variable is extended by adding a deterministic Qbit of value 0.
    - An unknown Qwhole variable is extended by adding a Qubit is superposition state, i.e. the value is set to S.

Also, it is possible to check the specific quantum bit level expression of an assignment, like Q bit level 2 in the following example, by seting *forBit* argument to *2*. The default value  of *forBit* argument is *dann5.d5o2.AllBits()*, which returns *-1*.

In [10]:
print(aA.toString(True, 2))


A2/1/ = a2/S/ + b2/S/ + #1/S/
#1/S/ = #[A1/1/]


- **NOTE**: The varaible *#[A1/1/]* represents a carryover of quantum level 1 assignment. In this example it is equal to a temporary carryover variable *#6/S/*, which is used in addition expresion for quantum bit level 2.

To solve the assignment we can use dann5.d5o2 built in quantum annealing simulator, by invoking *Qassignmnet solve()* method.

In [11]:
print(aA.solve())

A/5:15/; a/4:8/; b/3:7/
A/5:15/; a/4:12/; b/3:3/
A/5:15/; a/4:10/; b/3:5/
A/5:15/; a/4:14/; b/3:1/
A/5:15/; a/4:9/; b/3:6/
A/5:15/; a/4:13/; b/3:2/
A/5:15/; a/4:11/; b/3:4/
A/5:15/; a/4:15/; b/3:0/




The *solve()* method returns all possible solutions of an assignment within the bounds of defined variables, i.e. the unknown variables Qbits sizes.
- **NOTE**: 'a' = 8 and 'b' = 7 is a valid solution, but 'a' = 7 and 'b' = 8 is not as Qwhole variable 'b' is defined with only 3 Qbits. By carefully reviewing the returned results you will see that *8 <= 'a' <= 15* and *0 <= 'b' <= 7*, which are appropriate results considering that we set Q variable 'b' to have only 3 Qbits, i.e. imposing additional condition that 'b' cannot be bigger than 7.

> If you go back and change only the number of Qbits for variable 'b', the solution set for Q assignment aA will be different

## Quantum variable binding in dann5.d5o2

It is important to understand that defined quantum varaibles within an quantum expression once assigned are bound to the Qassignment.

In [12]:
print('Original variables:\n{} = {} + {}'.format(A.toString(), var.toString(), b.toString()))
print('Quantum assignment:\n{}'.format(aA.toString()))
print('Solutions:\n{}'.format(aA.solutions()))

Original variables:
A/4:15/ = a/4:U/ + b/3:U/
Quantum assignment:
A/5:15/ = (a/4:U/ + b/3:U/)
Solutions:
A/5:15/; a/4:8/; b/3:7/
A/5:15/; a/4:12/; b/3:3/
A/5:15/; a/4:10/; b/3:5/
A/5:15/; a/4:14/; b/3:1/
A/5:15/; a/4:9/; b/3:6/
A/5:15/; a/4:13/; b/3:2/
A/5:15/; a/4:11/; b/3:4/
A/5:15/; a/4:15/; b/3:0/




We can continue to use the quantum varaibles to find factors of number 15:  

In [13]:
eM = A.assign(var * b)
print('Solutions of\n{}\nare:\n{}'.format(eM.toString(),eM.solve()))

Solutions of
A/7:15/ = (a/4:U/ * b/3:U/)
are:
A/7:15/; a/4:15/; b/3:1/
A/7:15/; a/4:3/; b/3:5/
A/7:15/; a/4:5/; b/3:3/




- **NOTE**: Even though, it is possible to reuse same quantum varaibles for a different quantum assignments, the best practice is to *use different quantum variables for unrelated quantum assignments*!

In [14]:
c = Qwhole(4, 'c')
d = Qwhole(3, 'd')
M = Qwhole('M', 15)
unrelatedAssignment = M.assign(c * d)
print('Solutions of\n{}\nare:\n{}'.format(unrelatedAssignment.toString(),unrelatedAssignment.solve()))

Solutions of
M/7:15/ = (c/4:U/ * d/3:U/)
are:
M/7:15/; c/4:15/; d/3:1/
M/7:15/; c/4:3/; d/3:5/
M/7:15/; c/4:5/; d/3:3/




Construction of a *quantum rutine with more then one related assignment* might require use of same variabled across multiple assignments and/or expressions, however this is an advanced feature, which will be covered later.

## d5o 2 assignment conversion to QUBO

Now we can convert the Q assignments 'aA' or eM to their [QUBO](https://minatoyuichiro.medium.com/qubo-select-k-qubits-from-n-qubits-on-qaoa-651dca0a0e9b) presentation ([learn more about QUBO in context of DWave  Binary Quadratic Models (BQM)](https://docs.dwavesys.com/docs/latest/c_gs_3.html#qubo)). 
There are 2 forms of QUBO presentation that can be requested from a Q equation, **generic** and **finalized**. 
> **A generic QUBO** is transformation where all Qassignment operations and variables are converted into qubo presentation wether they are deterministic or unknown (with Qbits in supperposition state).

To retrieve a **generic QUBO** presentation of the Q assignment *aA* use *qubo* method with *finalized* argument set to *False*.

In [15]:
gQaA = aA.qubo(False)
print('Generic QUBO presentation of aA assignment:\n', gQaA)

from dann5.d5o2 import Qanalyzer
gQaAnlyzr = Qanalyzer(gQaA)
print("\nLinear nodes\n", gQaAnlyzr.nodes())
print("\nQuadratic branches\n", gQaAnlyzr.branches())

Generic QUBO presentation of aA assignment:
 {('#0', '#0'): 5.0, ('#0', '#1'): -4.0, ('#0', 'A1'): -2.0, ('#1', '#1'): 5.0, ('#1', '#2'): -4.0, ('#1', 'A2'): -2.0, ('#2', '#2'): 5.0, ('#2', 'A3'): -2.0, ('#2', 'A4'): -4.0, ('A0', '#0'): 4.0, ('A0', 'A0'): 1.0, ('A1', '#1'): 4.0, ('A1', 'A1'): 1.0, ('A2', '#2'): 4.0, ('A2', 'A2'): 1.0, ('A3', 'A3'): 1.0, ('A3', 'A4'): 4.0, ('A4', 'A4'): 4.0, ('a0', '#0'): -4.0, ('a0', 'A0'): -2.0, ('a0', 'a0'): 1.0, ('a0', 'b0'): 2.0, ('a1', '#0'): 2.0, ('a1', '#1'): -4.0, ('a1', 'A1'): -2.0, ('a1', 'a1'): 1.0, ('a1', 'b1'): 2.0, ('a2', '#1'): 2.0, ('a2', '#2'): -4.0, ('a2', 'A2'): -2.0, ('a2', 'a2'): 1.0, ('a2', 'b2'): 2.0, ('a3', '#2'): 2.0, ('a3', 'A3'): -2.0, ('a3', 'A4'): -4.0, ('a3', 'a3'): 1.0, ('b0', '#0'): -4.0, ('b0', 'A0'): -2.0, ('b0', 'b0'): 1.0, ('b1', '#0'): 2.0, ('b1', '#1'): -4.0, ('b1', 'A1'): -2.0, ('b1', 'b1'): 1.0, ('b2', '#1'): 2.0, ('b2', '#2'): -4.0, ('b2', 'A2'): -2.0, ('b2', 'b2'): 1.0}

Linear nodes
 [('#0', 5.0), ('#1', 5.0),

A Q assignment aA can be expressed as a QUBO problem defined using an upper-diagonal matrix gQaA expresed as a python 'dict' class, where keys are 'tuple' pairs of variables Qbit level names and the values of the elements are linear and quadratic coefficients. The key 'tuple' pairs with the same variable Qbit level name are **linear nodes**, while those those with diferent names are **quadratic branches**, as per DWave [Binary Quadratic Models](https://docs.dwavesys.com/docs/latest/c_gs_3.html#objective-functions).

> **A finalized QUBO** is a consolidated **generic QUBO** where deterministic variables are replaced with their values.

To retrieve a **finalized QUBO** presentation of the Q assignment *aA* use *qubo* method without arguments as *True* is a default value for *finalized* argument.

In [16]:
fQaA = aA.qubo()
print('Finalized QUBO presentation of aA assignment:\n', fQaA)
fQaAnlyzr = Qanalyzer(fQaA)
print("\nLinear nodes\n", fQaAnlyzr.nodes())
print("\nQuadratic branches\n", fQaAnlyzr.branches())

Finalized QUBO presentation of aA assignment:
 {('#0', '#0'): 7.0, ('#0', '#1'): -4.0, ('#1', '#1'): 7.0, ('#1', '#2'): -4.0, ('#2', '#2'): 7.0, ('a0', '#0'): -4.0, ('a0', 'a0'): -1.0, ('a0', 'b0'): 2.0, ('a1', '#0'): 2.0, ('a1', '#1'): -4.0, ('a1', 'a1'): -1.0, ('a1', 'b1'): 2.0, ('a2', '#1'): 2.0, ('a2', '#2'): -4.0, ('a2', 'a2'): -1.0, ('a2', 'b2'): 2.0, ('a3', '#2'): 2.0, ('a3', 'a3'): -1.0, ('b0', '#0'): -4.0, ('b0', 'b0'): -1.0, ('b1', '#0'): 2.0, ('b1', '#1'): -4.0, ('b1', 'b1'): -1.0, ('b2', '#1'): 2.0, ('b2', '#2'): -4.0, ('b2', 'b2'): -1.0}

Linear nodes
 [('#0', 7.0), ('#1', 7.0), ('#2', 7.0), ('a0', -1.0), ('a1', -1.0), ('a2', -1.0), ('a3', -1.0), ('b0', -1.0), ('b1', -1.0), ('b2', -1.0)]

Quadratic branches
 [(('#0', '#1'), -4.0), (('#1', '#2'), -4.0), (('a0', '#0'), -4.0), (('a0', 'b0'), 2.0), (('a1', '#0'), 2.0), (('a1', '#1'), -4.0), (('a1', 'b1'), 2.0), (('a2', '#1'), 2.0), (('a2', '#2'), -4.0), (('a2', 'b2'), 2.0), (('a3', '#2'), 2.0), (('b0', '#0'), -4.0), (('b1', '#

When we compare *generic and finanilized QUBO presentations* we see that *linear nodes* with deterministic values such are {A0 - A4} are removed as they are constants, while their *quadratic branches* through replacement of specific values can become *linear nodes* and they are consolidated with coresponding *linear nodes*.  

> To process a Q assignment 'aA' on a QAC, we need to create a **finalized QUBO** presentation of Q assignment.

## Solving Q assignment using DWave

To solve our problem statement *'A' = 'a' + 'b', where 'A' = 15 and 'a'='b'=U(nknown)* defined by Q equation *aA* we will use [D-Wave's Exact Solver](https://docs.ocean.dwavesys.com/projects/dimod/en/0.7.0/reference/generated/dimod.reference.samplers.ExactSolver.sample.html):

In [17]:
from dimod import ExactSolver
exactSolver = ExactSolver()                   # local solver

... and execute request *sample_qubo()* to solve *'fQaA'*, a **finalized QUBO of aA Q assignment**.

In [18]:
sampleset = exactSolver.sample_qubo(fQaA)

The **D-Wave sampleset has to be converted into a python dictionary** ('samples'), before it is passed to the Q equation *aA* to provide solutions in a human readable form: 

In [19]:
samples = [dict(sample) for sample in sampleset.lowest().samples()]
aA.reset()                      # removing old solutions
aA.add(samples)
print(aA.solutions())

A/5:15/; a/4:10/; b/3:5/
A/5:15/; a/4:11/; b/3:4/
A/5:15/; a/4:8/; b/3:7/
A/5:15/; a/4:9/; b/3:6/
A/5:15/; a/4:14/; b/3:1/
A/5:15/; a/4:15/; b/3:0/
A/5:15/; a/4:12/; b/3:3/
A/5:15/; a/4:13/; b/3:2/




> Note that we have used **sampleset.lowest().samples()** to ensure that from the whole sampleset of all posible solution of QUBO, we retrieve only those with the lowest energy level as the best solution for our *aA* Q assignment.

The same **finalized QUBO** can be executed on [quantum annealing computer (QAC)](https://docs.dwavesys.com/docs/latest/c_solver_parameters.html):

In [20]:
from dwave.system import DWaveSampler, EmbeddingComposite
qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus', 'qpu': True})
solver = EmbeddingComposite(qpu_advantage)
sampleset = solver.sample_qubo(fQaA)

In [21]:
samples = [dict(sample) for sample in sampleset.lowest().samples()]
aA.reset()
aA.add(samples)
print(aA.solutions())

A/5:15/; a/4:14/; b/3:1/




## Quantum assignments and expressions in d5o 2
In d5o we can create an **equal quantum expression** *eA* which has the same form as the **quantum assignment** *aA*.

In [22]:
eA = A == var + b
eA

- **NOTE**: eA is an instance of a dann5.d5o2.QwholeExpression class, which is a specilization of dann5.d5o2.Qexpression, with the following Qwhole addition and equal expression:

In [23]:
print(eA.toString())

((a/4:U/ + b/3:U/) == A/5:15/)


The Qbit level decomposition of the Q expression is bit more complex then in case of Q assignment.

In [24]:
print(eA.toString(True))


_+150/S/ == A0/1/
_+150/S/ = a0/S/ ^ b0/S/

_+151/S/ == A1/1/
_+151/S/ = a1/S/ + b1/S/ + #21/S/
#21/S/ = #[_+150/S/]

_+152/S/ == A2/1/
_+152/S/ = a2/S/ + b2/S/ + #22/S/
#22/S/ = #[_+151/S/]

_+153/S/ == A3/1/
_+153/S/ = a3/S/ ^ #23/S/
#23/S/ = #[_+152/S/]

_+154/S/ == A4/0/
_+154/S/ = #[_+153/S/]



In [25]:
eAnlyzr = Qanalyzer(eA.qubo())
print('Q expression A == a + b has {} nodes and {} branches'.format(eAnlyzr.nodesNo(), eAnlyzr.branchesNo()))
print('Q assignment A = a + b has {} nodes and {} branches'.format(fQaAnlyzr.nodesNo(), fQaAnlyzr.branchesNo()))

Q expression A == a + b has 15 nodes and 32 branches
Q assignment A = a + b has 10 nodes and 16 branches


However, the *eA* Q expression has same results as *aA* Q assignment.

In [26]:
print(eA.solve())

A/5:15/; _+15/5:15/; a/4:8/; b/3:7/
A/5:15/; _+15/5:15/; a/4:12/; b/3:3/
A/5:15/; _+15/5:15/; a/4:10/; b/3:5/
A/5:15/; _+15/5:15/; a/4:14/; b/3:1/
A/5:15/; _+15/5:15/; a/4:9/; b/3:6/
A/5:15/; _+15/5:15/; a/4:13/; b/3:2/
A/5:15/; _+15/5:15/; a/4:11/; b/3:4/
A/5:15/; _+15/5:15/; a/4:15/; b/3:0/



## Using Constants in a Q assignment
A definition and use of a constant in d5o is same as definition of any other variable with a deterministic value. In order to easily distinguish constants from variables we recommend following syntax as an example of best coding practice:

> _ _15 = Qvar("15__", 15)

Based on above we can make a Q whole definition such as:

In [27]:
_15 = Qwhole("15_", 15)